In [1]:
#######################################################################################################
# Summary
# 1. PyTorch Multi-GPU example
# 2. On-the-fly data-augmentation (random crop, random flip)
#######################################################################################################

In [2]:
MULTI_GPU = True

In [3]:
import os
import sys
import time
import multiprocessing
import numpy as np
import pandas as pd
import torch
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics.ranking import roc_auc_score
from sklearn.model_selection import train_test_split
from PIL import Image
from common.utils import download_data_chextxray, get_imgloc_labels, get_train_valid_test_split
from common.utils import compute_roc_auc, get_cuda_version, get_cudnn_version, get_gpu_name
from common.utils import yield_mb
from common.params_dense import *

In [4]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("PyTorch: ", torch.__version__)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
PyTorch:  0.3.1.post2
Numpy:  1.14.2
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 9.0.176
CuDNN Version  7.0.5


In [5]:
CPU_COUNT = multiprocessing.cpu_count()
GPU_COUNT = len(get_gpu_name())
print("CPUs: ", CPU_COUNT)
print("GPUs: ", GPU_COUNT)

CPUs:  24
GPUs:  4


In [6]:
# Model-params
IMAGENET_RGB_MEAN_TORCH = [0.485, 0.456, 0.406]
IMAGENET_RGB_SD_TORCH = [0.229, 0.224, 0.225]
# Paths
CSV_DEST = "chestxray"
IMAGE_FOLDER = os.path.join(CSV_DEST, "images")
LABEL_FILE = os.path.join(CSV_DEST, "Data_Entry_2017.csv")
print(IMAGE_FOLDER, LABEL_FILE)

chestxray/images chestxray/Data_Entry_2017.csv


In [7]:
# Manually scale to multi-gpu
if MULTI_GPU:
    assert torch.cuda.is_available()
     # enables cudnn's auto-tuner
    torch.backends.cudnn.benchmark=True
    LR *= GPU_COUNT 
    BATCHSIZE *= GPU_COUNT

In [8]:
%%time
# Download data
# Wall time: 17min 58s
print("Please make sure to download")
print("https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy")
download_data_chextxray(CSV_DEST)

Please make sure to download
https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
Data already exists
CPU times: user 758 ms, sys: 288 ms, total: 1.05 s
Wall time: 1.05 s


In [9]:
#####################################################################################################
## Data Loading

In [10]:
# Normalise by imagenet mean/sd
normalize = transforms.Normalize(IMAGENET_RGB_MEAN_TORCH,
                                 IMAGENET_RGB_SD_TORCH)

In [11]:
class XrayData(Dataset):
    def __init__(self, img_dir, lbl_file, patient_ids, transform=None):
        
        self.img_locs, self.labels = get_imgloc_labels(img_dir, lbl_file, patient_ids)
        self.transform = transform
        print("Loaded {} labels and {} images".format(len(self.labels), len(self.img_locs)))
    
    def __getitem__(self, idx):
        im_file = self.img_locs[idx]
        im_rgb = Image.open(im_file)
        label = self.labels[idx]
        if self.transform is not None:
            im_rgb = self.transform(im_rgb)
        return im_rgb, torch.FloatTensor(label)
        
    def __len__(self):
        return len(self.img_locs)

In [12]:
def no_augmentation_dataset(img_dir, lbl_file, patient_ids, normalize):
    dataset = XrayData(img_dir, lbl_file, patient_ids,
                       transform=transforms.Compose([
                           transforms.Resize(WIDTH),
                           transforms.ToTensor(),  
                           normalize]))
    return dataset

In [13]:
train_set, valid_set, test_set = get_train_valid_test_split(TOT_PATIENT_NUMBER)

train:21563 valid:3080 test:6162


In [14]:
# Dataset for training
train_dataset = XrayData(img_dir=IMAGE_FOLDER,
                         lbl_file=LABEL_FILE,
                         patient_ids=train_set,
                         transform=transforms.Compose([
                             transforms.RandomResizedCrop(size=WIDTH),
                             transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(),  # need to convert image to tensor!
                             normalize]))

Loaded 87306 labels and 87306 images


In [15]:
valid_dataset = no_augmentation_dataset(IMAGE_FOLDER, LABEL_FILE, valid_set, normalize)
test_dataset = no_augmentation_dataset(IMAGE_FOLDER, LABEL_FILE, test_set, normalize)

Loaded 7616 labels and 7616 images
Loaded 17198 labels and 17198 images


In [16]:
#####################################################################################################
## Helper Functions

In [17]:
def get_symbol(out_features=CLASSES, multi_gpu=MULTI_GPU):
    model = models.densenet.densenet121(pretrained=True)
    # Replace classifier (FC-1000) with (FC-14)
    model.classifier = nn.Sequential(
        nn.Linear(model.classifier.in_features, out_features), 
        nn.Sigmoid())
    if multi_gpu:
        model = nn.DataParallel(model)
    # CUDA
    model.cuda()  
    return model

In [18]:
def init_symbol(sym, lr=LR):
    opt = optim.Adam(sym.parameters(), lr=lr, betas=(0.9, 0.999))
    # BCE Loss since classes not mutually exclusive + Sigmoid FC-layer
    cri = nn.BCELoss()
    sch = ReduceLROnPlateau(opt, factor=0.1, patience=5, mode='min')
    return opt, cri, sch 

In [19]:
def train_epoch(model, dataloader, optimizer, criterion, epoch, batch=BATCHSIZE):
    model.train()
    print("Training epoch {}".format(epoch+1))
    loss_val = 0
    loss_cnt = 0
    for data, target in dataloader:
        # Get samples
        data = Variable(torch.FloatTensor(data).cuda())
        target = Variable(torch.FloatTensor(target).cuda())
        # Init
        optimizer.zero_grad()
        # Forwards
        output = model(data)
        # Loss
        loss = criterion(output, target)
        # Back-prop
        loss.backward()
        optimizer.step()   
         # Log the loss
        loss_val += loss.data[0]
        loss_cnt += 1
    print("Training loss: {0:.4f}".format(loss_val/loss_cnt))

In [20]:
def valid_epoch(model, dataloader, criterion, epoch, phase='valid', 
                batch=BATCHSIZE, cl=CLASSES):
    model.eval()
    if phase == 'testing':
        print("Testing epoch {}".format(epoch+1))
    else:
        print("Validating epoch {}".format(epoch+1))
    out_pred = torch.FloatTensor().cuda()
    out_gt = torch.FloatTensor().cuda()
    loss_val = 0
    loss_cnt = 0
    for data, target in dataloader:
        # Get samples
        data = Variable(torch.FloatTensor(data).cuda(), volatile=True)
        target = Variable(torch.FloatTensor(target).cuda(), volatile=True)
         # Forwards
        output = model(data)
        # Loss
        loss = criterion(output, target)
        # Log the loss
        loss_val += loss.data[0]
        loss_cnt += 1
        # Log for AUC
        out_pred = torch.cat((out_pred, output.data), 0)
        out_gt = torch.cat((out_gt, target.data), 0)
        
    loss_mean = loss_val/loss_cnt
    out_gt = out_gt.cpu().numpy()
    out_pred = out_pred.cpu().numpy()    
    
    if phase == 'testing':
        print("Test-Dataset loss: {0:.4f}".format(loss_mean))
        print("Test-Dataset AUC: {0:.4f}".format(compute_roc_auc(out_gt, out_pred, cl)))
    else:
        print("Validation loss: {0:.4f}".format(loss_mean))
        print("Validation AUC: {0:.4f}".format(compute_roc_auc(out_gt, out_pred, cl)))
    return loss_mean

In [21]:
#def print_learning_rate(opt):
#    for param_group in opt.param_groups:
#        print("Learning rate: ", param_group['lr'])

In [22]:
# DataLoaders
# pin_memory brings errors on 4 GPUs
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCHSIZE,
                          shuffle=True, num_workers=CPU_COUNT)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=16*BATCHSIZE,
                          shuffle=False, num_workers=CPU_COUNT)
test_loader = DataLoader(dataset=test_dataset, batch_size=16*BATCHSIZE,
                         shuffle=False, num_workers=CPU_COUNT)

In [23]:
#####################################################################################################
## Train CheXNet

In [24]:
%%time
# Load symbol
chexnet_sym = get_symbol()

CPU times: user 2.15 s, sys: 1.12 s, total: 3.27 s
Wall time: 3.29 s


In [25]:
%%time
# Load optimiser, loss
optimizer, criterion, scheduler = init_symbol(chexnet_sym)

CPU times: user 2.26 ms, sys: 0 ns, total: 2.26 ms
Wall time: 2.27 ms


In [26]:
%%time
# 1 GPU - Main training loop: 41min 7s
# 4 GPU - Main training loop: 16min 45s
loss_min = float("inf")    
# Main train/val loop
for j in range(EPOCHS):
    stime = time.time()
    train_epoch(chexnet_sym, train_loader, optimizer, criterion, j)
    loss_val = valid_epoch(chexnet_sym, valid_loader, criterion, j)
    # LR Schedule
    scheduler.step(loss_val)
    #print_learning_rate(optimizer)
    
    # I comment this out to create a fair test against Keras
    # Keras cannot checkpoint multi-gpu models at the moment
    
    #if loss_val < loss_min:
    #    print("Loss decreased. Saving ...")
    #    loss_min = loss_val
    #    torch.save({'epoch': j + 1, 
    #                'state_dict': chexnet_sym.state_dict(), 
    #                'best_loss': loss_min, 
    #                'optimizer' : optimizer.state_dict()}, 'best_chexnet.pth.tar')
    
    etime = time.time()
    print("Epoch time: {0:.0f} seconds".format(etime-stime))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

Training epoch 1
Training loss: 0.1726
Validating epoch 1
Validation loss: 0.1470
Full AUC [0.7827262054948135, 0.8678967857711399, 0.7868547413012692, 0.8992170143507066, 0.8917194602381643, 0.8857012338275809, 0.7189545955150085, 0.7053378911385748, 0.6496618519658628, 0.7558621524717974, 0.6854691633675443, 0.7658281617973451, 0.7675622713999648, 0.8501016713428744]
Validation AUC: 0.7866
Epoch time: 245 seconds
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Training epoch 2
Training loss: 0.1583
Validating epoch 2
Validation loss: 0.1436
Full AUC [0.784890576815251, 0.882323912068192, 0.7823860155608633, 0.9101201447161874, 0.8948739945316595, 0.8831696527066704, 0.7705390269240082, 0.8127793846962925, 0.6550024281377378, 0.813639900352265, 0.723578406539455, 0.7864047339111591, 0.7823377484674239, 0.8725379537993049]
Validation AUC: 0.8110
Epoch time: 195 seconds
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Training epoch 3
Training loss: 0.1557
Validating epoch 3
Validation l

In [27]:
#####################################################################################################
## Test CheXNet

In [28]:
%%time
# Load model for testing
# I comment this out to create a fair test against Keras
#chexnet_sym_test = get_symbol()
#chkpt = torch.load("best_chexnet.pth.tar")
#chexnet_sym_test.load_state_dict(chkpt['state_dict'])

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10.5 µs


In [29]:
%%time
## Evaluate: 37s
# AUC: 0.8179
#test_loss = valid_epoch(chexnet_sym_test, test_loader, criterion, -1, 'testing')
test_loss = valid_epoch(chexnet_sym, test_loader, criterion, -1, 'testing')

Testing epoch 0
Test-Dataset loss: 0.1511
Full AUC [0.8151923157000877, 0.8759298579325923, 0.8046419265823063, 0.8941065070796208, 0.8885609803069663, 0.9194565806582362, 0.7133234576650442, 0.8601636832768909, 0.6237482379558829, 0.8441295028575039, 0.7526728274292551, 0.801820880646043, 0.7669334058813965, 0.8902362513366339]
Test-Dataset AUC: 0.8179
CPU times: user 7.13 s, sys: 5.93 s, total: 13.1 s
Wall time: 37.2 s


In [30]:
#####################################################################################################
## Synthetic Data (Pure Training)

In [31]:
# Test on fake-data -> no IO lag
batch_in_epoch = len(train_dataset.labels)//BATCHSIZE
tot_num = batch_in_epoch * BATCHSIZE
print(tot_num)

87296


In [32]:
fake_X = np.random.rand(tot_num, 3, 224, 224).astype(np.float32)
fake_y = np.random.rand(tot_num, CLASSES).astype(np.float32) 

In [33]:
%%time
# 4 GPU - Synthetic data: 11min 37s
for j in range(EPOCHS):
    train_epoch(chexnet_sym, 
                yield_mb(fake_X, fake_y, BATCHSIZE, shuffle=False),
                optimizer, 
                criterion,
                j)

Training epoch 1
Training loss: 0.7126
Training epoch 2
Training loss: 0.6933
Training epoch 3
Training loss: 0.6933
Training epoch 4
Training loss: 0.6933
Training epoch 5
Training loss: 0.6933
CPU times: user 13min 48s, sys: 1min 50s, total: 15min 38s
Wall time: 11min 37s
